# Generación de Variables de Riesgo - Todos los Usuarios
## Home Credit Default Risk

**Objetivo**: Crear un DataFrame con todas las variables de riesgo para los 307,511 usuarios

---

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

print("✓ Librerías cargadas")

## 1. Carga de Datos

In [ ]:
print("Cargando datasets...\n")

app_train = pd.read_csv('home-credit-default-risk/application_train.csv')
bureau = pd.read_csv('home-credit-default-risk/bureau.csv')
bureau_balance = pd.read_csv('home-credit-default-risk/bureau_balance.csv')
prev_app = pd.read_csv('home-credit-default-risk/previous_application.csv')
pos_cash = pd.read_csv('home-credit-default-risk/POS_CASH_balance.csv')
credit_card = pd.read_csv('home-credit-default-risk/credit_card_balance.csv')
installments = pd.read_csv('home-credit-default-risk/installments_payments.csv')

print(f"✓ Application Train: {app_train.shape[0]:,} usuarios")
print(f"✓ Bureau: {bureau.shape[0]:,} registros")
print(f"✓ Bureau Balance: {bureau_balance.shape[0]:,} registros")
print(f"✓ Previous Application: {prev_app.shape[0]:,} registros")
print(f"✓ POS Cash: {pos_cash.shape[0]:,} registros")
print(f"✓ Credit Card: {credit_card.shape[0]:,} registros")
print(f"✓ Installments: {installments.shape[0]:,} registros")
print(f"\n✓ Datos cargados correctamente")

## 2. Función de Extracción de Variables

Esta función extrae **40 variables de riesgo** para cada usuario.

In [ ]:
def extraer_perfil_riesgo(user_id):
    """
    Extrae 40 variables de riesgo para un usuario.
    
    Incluye:
    - Crédito ocupado (TOTAL_DEUDA_ACTUAL)
    - Tasa de interés (TASA_INTERES_PROMEDIO)
    - Créditos en impago (CREDITOS_CON_IMPAGO)
    - Ratio pago/cuota (RATIO_PAGO_CUOTA)
    - Ratio pago/mínimo tarjetas (RATIO_PAGO_MINIMO_TC)
    """
    
    user_app = app_train[app_train['SK_ID_CURR'] == user_id].iloc[0]
    
    perfil = {
        'SK_ID_CURR': user_id,
        'TARGET': user_app['TARGET']
    }
    
    # MONTO DEL CRÉDITO
    perfil['AMT_CREDIT'] = user_app['AMT_CREDIT']
    perfil['AMT_ANNUITY'] = user_app['AMT_ANNUITY']
    
    # EDAD
    days_birth = user_app['DAYS_BIRTH']
    perfil['EDAD_ANOS'] = abs(days_birth) / 365.25
    perfil['DAYS_BIRTH'] = days_birth
    
    # HISTORIAL BURÓ
    user_bureau = bureau[bureau['SK_ID_CURR'] == user_id]
    perfil['ES_PRIMER_CREDITO'] = len(user_bureau) == 0
    perfil['CANTIDAD_CREDITOS_BURO'] = len(user_bureau)
    
    # SCORES EXTERNOS
    perfil['EXT_SOURCE_1'] = user_app['EXT_SOURCE_1']
    perfil['EXT_SOURCE_2'] = user_app['EXT_SOURCE_2']
    perfil['EXT_SOURCE_3'] = user_app['EXT_SOURCE_3']
    
    scores = [perfil['EXT_SOURCE_1'], perfil['EXT_SOURCE_2'], perfil['EXT_SOURCE_3']]
    scores_validos = [s for s in scores if pd.notna(s)]
    perfil['SCORE_PROMEDIO'] = np.mean(scores_validos) if scores_validos else np.nan
    
    # VARIABLES DE BUREAU
    if len(user_bureau) > 0:
        perfil['TOTAL_CREDITO_DISPONIBLE'] = user_bureau['AMT_CREDIT_SUM_LIMIT'].sum()
        perfil['TOTAL_CREDITO_OTORGADO'] = user_bureau['AMT_CREDIT_SUM'].sum()
        perfil['TOTAL_DEUDA_ACTUAL'] = user_bureau['AMT_CREDIT_SUM_DEBT'].sum()  # CRÉDITO OCUPADO
        perfil['CREDITOS_ACTIVOS'] = (user_bureau['CREDIT_ACTIVE'] == 'Active').sum()
        perfil['CREDITOS_CERRADOS'] = (user_bureau['CREDIT_ACTIVE'] == 'Closed').sum()
        perfil['MAX_DIAS_MORA'] = user_bureau['CREDIT_DAY_OVERDUE'].max()
        perfil['TIENE_IMPAGOS'] = perfil['MAX_DIAS_MORA'] > 0
        
        # Bureau balance
        bureau_ids = user_bureau['SK_ID_BUREAU'].tolist()
        user_bureau_balance = bureau_balance[bureau_balance['SK_ID_BUREAU'].isin(bureau_ids)]
        
        if len(user_bureau_balance) > 0:
            estados_mora = user_bureau_balance['STATUS'].isin(['1', '2', '3', '4', '5'])
            perfil['MESES_CON_MORA'] = estados_mora.sum()
            perfil['PCT_MESES_MORA'] = (perfil['MESES_CON_MORA'] / len(user_bureau_balance)) * 100
            perfil['CREDITOS_CON_IMPAGO'] = user_bureau_balance.groupby('SK_ID_BUREAU')['STATUS'].apply(
                lambda x: x.isin(['1','2','3','4','5']).any()
            ).sum()
        else:
            perfil['MESES_CON_MORA'] = 0
            perfil['PCT_MESES_MORA'] = 0
            perfil['CREDITOS_CON_IMPAGO'] = 0
    else:
        perfil.update({
            'TOTAL_CREDITO_DISPONIBLE': 0,
            'TOTAL_CREDITO_OTORGADO': 0,
            'TOTAL_DEUDA_ACTUAL': 0,
            'CREDITOS_ACTIVOS': 0,
            'CREDITOS_CERRADOS': 0,
            'MAX_DIAS_MORA': 0,
            'TIENE_IMPAGOS': False,
            'MESES_CON_MORA': 0,
            'PCT_MESES_MORA': 0,
            'CREDITOS_CON_IMPAGO': 0
        })
    
    # CONSULTAS BURÓ
    bureau_cols = ['AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 
                   'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON',
                   'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']
    perfil['TOTAL_CONSULTAS_BURO'] = sum([user_app[col] for col in bureau_cols if pd.notna(user_app[col])])
    
    # INGRESOS
    perfil['AMT_INCOME_TOTAL'] = user_app['AMT_INCOME_TOTAL']
    perfil['NAME_EDUCATION_TYPE'] = user_app['NAME_EDUCATION_TYPE']
    perfil['CREDIT_INCOME_RATIO'] = perfil['AMT_CREDIT'] / perfil['AMT_INCOME_TOTAL']
    
    # GASTOS Y DEPENDIENTES
    perfil['NAME_FAMILY_STATUS'] = user_app['NAME_FAMILY_STATUS']
    perfil['CNT_CHILDREN'] = user_app['CNT_CHILDREN']
    perfil['CNT_FAM_MEMBERS'] = user_app['CNT_FAM_MEMBERS']
    perfil['CODE_GENDER'] = user_app['CODE_GENDER']
    perfil['INGRESO_PER_CAPITA'] = (perfil['AMT_INCOME_TOTAL'] / perfil['CNT_FAM_MEMBERS'] 
                                     if perfil['CNT_FAM_MEMBERS'] > 0 else perfil['AMT_INCOME_TOTAL'])
    
    # PRÉSTAMOS PREVIOS
    user_prev_app = prev_app[prev_app['SK_ID_CURR'] == user_id]
    perfil['NUM_PRESTAMOS_PREVIOS'] = len(user_prev_app)
    
    if len(user_prev_app) > 0:
        perfil['TASA_INTERES_PROMEDIO'] = user_prev_app['RATE_INTEREST_PRIMARY'].mean()  # TASA DE INTERÉS
        perfil['PLAZO_PROMEDIO'] = user_prev_app['CNT_PAYMENT'].mean()
        perfil['MONTO_PROMEDIO_PREVIO'] = user_prev_app['AMT_CREDIT'].mean()
        perfil['TOTAL_CREDITO_HISTORICO'] = user_prev_app['AMT_CREDIT'].sum()
    else:
        perfil.update({
            'TASA_INTERES_PROMEDIO': np.nan,
            'PLAZO_PROMEDIO': np.nan,
            'MONTO_PROMEDIO_PREVIO': 0,
            'TOTAL_CREDITO_HISTORICO': 0
        })
    
    # ACTIVOS
    perfil['FLAG_OWN_CAR'] = user_app['FLAG_OWN_CAR']
    perfil['FLAG_OWN_REALTY'] = user_app['FLAG_OWN_REALTY']
    perfil['NUM_ACTIVOS'] = (perfil['FLAG_OWN_CAR'] == 'Y') + (perfil['FLAG_OWN_REALTY'] == 'Y')
    
    # COMPORTAMIENTO DE PAGO - INSTALLMENTS (RATIO PAGO/CUOTA)
    user_installments = installments[installments['SK_ID_CURR'] == user_id]
    if len(user_installments) > 0:
        valid_payments = user_installments[user_installments['AMT_INSTALMENT'] > 0].copy()
        if len(valid_payments) > 0:
            valid_payments['PAYMENT_RATIO'] = valid_payments['AMT_PAYMENT'] / valid_payments['AMT_INSTALMENT']
            perfil['RATIO_PAGO_CUOTA'] = valid_payments['PAYMENT_RATIO'].mean()
        else:
            perfil['RATIO_PAGO_CUOTA'] = np.nan
    else:
        perfil['RATIO_PAGO_CUOTA'] = np.nan
    
    # COMPORTAMIENTO DE PAGO - TARJETAS (RATIO PAGO/MÍNIMO)
    user_cc = credit_card[credit_card['SK_ID_CURR'] == user_id]
    if len(user_cc) > 0:
        valid_cc = user_cc[user_cc['AMT_INST_MIN_REGULARITY'] > 0].copy()
        if len(valid_cc) > 0:
            valid_cc['PAYMENT_MIN_RATIO'] = valid_cc['AMT_PAYMENT_CURRENT'] / valid_cc['AMT_INST_MIN_REGULARITY']
            perfil['RATIO_PAGO_MINIMO_TC'] = valid_cc['PAYMENT_MIN_RATIO'].mean()
        else:
            perfil['RATIO_PAGO_MINIMO_TC'] = np.nan
    else:
        perfil['RATIO_PAGO_MINIMO_TC'] = np.nan
    
    return perfil

## 3. Procesamiento de Todos los Usuarios

Generamos el DataFrame completo con todas las variables para los 307,511 usuarios:

In [ ]:
print("="*80)
print("GENERANDO DATASET COMPLETO")
print("="*80)
print(f"\nTotal de usuarios: {len(app_train):,}")
print(f"Variables por usuario: 40\n")

perfiles = []
errores = []

for user_id in tqdm(app_train['SK_ID_CURR'].values, desc="Procesando usuarios"):
    try:
        perfil = extraer_perfil_riesgo(user_id)
        perfiles.append(perfil)
    except Exception as e:
        errores.append((user_id, str(e)))
        continue

# Crear DataFrame final
df = pd.DataFrame(perfiles)

print(f"\n{'='*80}")
print("✅ DATASET COMPLETO GENERADO")
print(f"{'='*80}")
print(f"Usuarios procesados: {len(df):,}")
print(f"Variables: {len(df.columns)}")
print(f"Shape: {df.shape}")
if errores:
    print(f"⚠️  Errores: {len(errores)}")
print(f"{'='*80}\n")

## 4. Vista Previa del Dataset

In [ ]:
print("📊 Primeras 10 filas del dataset:\n")
df.head(10)

## 5. Información del Dataset

In [ ]:
print("\n📋 INFORMACIÓN DEL DATASET")
print("="*80)
print(f"\nColumnas ({len(df.columns)}):")
print(df.columns.tolist())

print(f"\nTipos de datos:")
print(df.dtypes.value_counts())

print(f"\nMemoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\nDistribución del TARGET:")
print(df['TARGET'].value_counts())
print(f"\nTasa de default: {df['TARGET'].mean() * 100:.2f}%")

## 6. Estadísticas Descriptivas

In [ ]:
print("\n📊 Estadísticas de variables numéricas:\n")
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols].describe()

## 7. Valores Faltantes

In [ ]:
print("\n📉 Porcentaje de valores faltantes por variable:\n")
missing_pct = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
missing_pct[missing_pct > 0]

## 8. Análisis por TARGET

In [ ]:
print("\n📊 COMPARACIÓN: Pagó (0) vs Default (1)")
print("="*80)

vars_comparar = [
    'EDAD_ANOS', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'CREDIT_INCOME_RATIO',
    'SCORE_PROMEDIO', 'TOTAL_DEUDA_ACTUAL', 'CREDITOS_ACTIVOS',
    'PCT_MESES_MORA', 'CREDITOS_CON_IMPAGO', 'NUM_ACTIVOS',
    'RATIO_PAGO_CUOTA', 'RATIO_PAGO_MINIMO_TC'
]

vars_comparar = [v for v in vars_comparar if v in df.columns]

comparacion = df.groupby('TARGET')[vars_comparar].mean().T
comparacion.columns = ['Pagó (0)', 'Default (1)']
comparacion['Diferencia'] = comparacion['Default (1)'] - comparacion['Pagó (0)']
comparacion['% Cambio'] = (comparacion['Diferencia'] / comparacion['Pagó (0)']) * 100

print(comparacion)
print("\n" + "="*80)

## 9. Guardar Dataset Final

In [ ]:
# Guardar a CSV
output_file = 'dataset_variables_completo.csv'
df.to_csv(output_file, index=False)

print("\n" + "="*80)
print("💾 DATASET GUARDADO")
print("="*80)
print(f"Archivo: {output_file}")
print(f"Usuarios: {len(df):,}")
print(f"Variables: {len(df.columns)}")
print(f"Tamaño: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("="*80)

print("\n✅ PROCESO COMPLETADO")
print("\nEl DataFrame 'df' contiene todas las variables para todos los usuarios.")
print(f"Puedes accederlo con: df")

---

## Resumen de Variables Generadas

### Total: 40 variables

**1. Identificación (2)**
- SK_ID_CURR
- TARGET

**2. Crédito Actual (2)**
- AMT_CREDIT
- AMT_ANNUITY

**3. Demografía (4)**
- EDAD_ANOS
- DAYS_BIRTH
- ES_PRIMER_CREDITO
- CANTIDAD_CREDITOS_BURO

**4. Scores Crediticios (4)**
- EXT_SOURCE_1
- EXT_SOURCE_2
- EXT_SOURCE_3
- SCORE_PROMEDIO

**5. Deuda e Historial (10)**
- TOTAL_CREDITO_DISPONIBLE
- TOTAL_CREDITO_OTORGADO
- **TOTAL_DEUDA_ACTUAL** (Crédito ocupado)
- CREDITOS_ACTIVOS
- CREDITOS_CERRADOS
- MAX_DIAS_MORA
- TIENE_IMPAGOS
- MESES_CON_MORA
- PCT_MESES_MORA
- **CREDITOS_CON_IMPAGO** (Cantidad de créditos con impago)

**6. Consultas (1)**
- TOTAL_CONSULTAS_BURO

**7. Ingresos (3)**
- AMT_INCOME_TOTAL
- NAME_EDUCATION_TYPE
- CREDIT_INCOME_RATIO

**8. Familia (5)**
- NAME_FAMILY_STATUS
- CNT_CHILDREN
- CNT_FAM_MEMBERS
- CODE_GENDER
- INGRESO_PER_CAPITA

**9. Préstamos Previos (5)**
- NUM_PRESTAMOS_PREVIOS
- **TASA_INTERES_PROMEDIO** (Tasa de interés)
- PLAZO_PROMEDIO
- MONTO_PROMEDIO_PREVIO
- TOTAL_CREDITO_HISTORICO

**10. Activos (3)**
- FLAG_OWN_CAR
- FLAG_OWN_REALTY
- NUM_ACTIVOS

**11. 🆕 Comportamiento de Pago (2)**
- **RATIO_PAGO_CUOTA** (Pago real / Pago esperado)
- **RATIO_PAGO_MINIMO_TC** (Pago real / Pago mínimo en tarjetas)

---

### Variables Clave Solicitadas:

✅ **Crédito ocupado**: TOTAL_DEUDA_ACTUAL  
✅ **Créditos en impago**: CREDITOS_CON_IMPAGO  
✅ **Tasa de interés**: TASA_INTERES_PROMEDIO  
✅ **Pagos/Pago mínimo**: RATIO_PAGO_CUOTA y RATIO_PAGO_MINIMO_TC

---

**Última ejecución**: 2025-11-25  
**Autores**: Gerardo Guerrero, Juan Pablo Cordero, Jerónimo Deli, Romain S